In [ ]:
from langgraph.store.base import BaseStore
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage, merge_message_runs
from langchain_core.runnables import RunnableConfig, config
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END, MessagesState
from pydantic import BaseModel, Field
from typing import Optional, List




In [ ]:
class Item(BaseModel):
    """Item information for user.
    
    IMPORTANT: Use null for unknown fields, not the string "None".
    Only populate fields with actual information from the conversation.
    """
    
    items: Optional[List[str]] = Field(
        default=None, 
        description="List of items in customer's order. Use null if no items ordered yet."
    )

In [ ]:
async def add_to_cart(state:MessagesState, config:RunnableConfig, store:BaseStore):
    """Add items to the user's cart."""

    global store 
    user_id= config["configurable"]["user_id"]

    last_message = state["messages"][-1]
    tool_call= last_message.tool_calls[0]
    args= tool_call.get('args', {})

    cart_items= args.get("cart_items", [])

    if not cart_items:

        return {
            "messages": [{
                "role": "tool",
                "content": "No items specified to add to cart.",
                "tool_call_id": tool_call['id']
            }]
        }
    
    try:
        pool = await get_pool()
        
        async with pool.connection() as conn:
            # Get current cart
            result = await conn.execute(
                "SELECT cart FROM user_profiles WHERE user_id = %s",
                (user_id,)
            )
            row = await result.fetchone()
            
            current_cart = row[0] if row and row[0] else []
            
            # Add new items to cart
            updated_cart = current_cart + cart_items
            
            # Update database
            await conn.execute(
                '''
                INSERT INTO user_profiles (user_id, cart, updated_at)
                VALUES (%s, %s, NOW())
                ON CONFLICT (user_id) DO UPDATE SET
                    cart = EXCLUDED.cart,
                    updated_at = NOW()
                ''',
                (user_id, updated_cart)
            )
            
            # Also update LangGraph store for chatbot memory
            namespace = ("profile", user_id)
            existing_memory = await store.aget(namespace, "user_memory")
            
            if existing_memory:
                memory_data = existing_memory.value
                memory_data['cart'] = updated_cart
            else:
                memory_data = {'cart': updated_cart}
            
            await store.aput(namespace, "user_memory", memory_data)
        
        # Format success message
        items_str = ", ".join(cart_items)
        return {
            "messages": [{
                "role": "tool",
                "content": f"Successfully added to cart: {items_str}. Current cart has {len(updated_cart)} item(s).",
                "tool_call_id": tool_call['id']
            }]
        }
    
    except Exception as e:
        logger.error(f"Failed to add to cart: {e}")
        return {
            "messages": [{
                "role": "tool",
                "content": "Failed to add items to cart. Please try again.",
                "tool_call_id": tool_call['id']
            }]
        }




def remove_item(delete_item, items):
    for i in delete_item:
        if i in items:
            items[i] -= 1
            if items[i] <= 0:
                del(items[i])
        
        else:
            print(f"{i} not found in cart")
    return items


def delete_total(delete_item, items):
    for i in delete_item:
        if i in items:
            del(items[i])
    return items

NameError: name 'MessagesState' is not defined